In [1]:
# datasource.py

from ast import Name
import random
from datetime import datetime, timedelta
import pandas as pd
import sqlite3

# Function to get the start date of a specific week in a given year
def get_week_start(year, week):
    return datetime.strptime(f'{year}-W{str(week).zfill(2)}-1', "%Y-W%W-%w")

# Function to generate order data
def generate_orders():
    order_ids = [f'2024-{str(i).zfill(2)}' for i in range(1, 51)]
    orders = []
    
    for order_id in order_ids:
        order_week = int(order_id.split('-')[1])
        order_week_start = get_week_start(2024, order_week)
        
        # Generate random order and delivery dates
        order_date = order_week_start - timedelta(weeks=random.choice([1, 2]), days=random.randint(0, 6))
        delivery_date = order_week_start + timedelta(weeks=random.choice([4, 5]), days=random.randint(0, 6))
        factory = random.choice(['China', 'Vietnam'])
        
        orders.append({
            'Order ID': order_id,
            'Order Date': order_date.strftime('%Y-%m-%d'),
            'Delivery Date': delivery_date.strftime('%Y-%m-%d'),
            'Factory': factory
        })
    return orders

In [2]:
# Function to generate sales data
def generate_sales_orders(orders):
    sales_ids = ['Sales A', 'Sales B', 'Sales C', 'Sales D', 'Sales E']
    sales_names = ['Alice', 'Bob', 'Charlie', 'David', 'Eve']
    customers = [f'Customer {i}' for i in range(1, 21)]
    
    data = []
    
    for sales_id, sales_name in zip(sales_ids, sales_names):
        assigned_customers = random.sample(customers, k=5)
        for customer in assigned_customers:
            num_orders = random.randint(3, 4)
            for _ in range(num_orders):
                order_id = random.choice([order['Order ID'] for order in orders])
                yield_rate = round(random.uniform(96.1, 99.8), 2)
                thru_put = round(random.uniform(950, 1200), 2)
                
                # Find the matching order details
                order_details = next((order for order in orders if order['Order ID'] == order_id), None)
                
                if order_details:
                    data.append([
                        sales_id, 
                        sales_name, 
                        customer, 
                        order_id, 
                        yield_rate, 
                        thru_put, 
                        order_details['Order Date'], 
                        order_details['Delivery Date'], 
                        order_details['Factory']
                    ])
    return data

In [3]:
# Function to save data to CSV
def save_to_csv(data, filename='sales_orders.csv'):
    columns = ['Sales ID', 'Sales Name', 'Customer ID', 'Order ID', 'Yield Rate', 
               'Thru_put', 'Order Date', 'Delivery Date', 'Factory']
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(filename, index=False)
    print(f"CSV file '{filename}' has been created successfully.")
    return df

In [5]:
import sqlite3

# Function to save data to SQLite database one by one
def save_to_sqlite(df, db_filename='sales_order1.db'):
    print("Saving data to SQLite database record by record...")
    conn = sqlite3.connect(db_filename)
    table_name = 'sales_orders1'

    # Create the table with an auto-incrementing 'id' field
    create_table_query = f'''
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        "Sales ID" TEXT,
        "Sales Name" TEXT,
        "Customer ID" TEXT,
        "Order ID" TEXT,
        "Yield Rate" REAL,
        "Thru_put" REAL,
        "Order Date" TEXT,
        "Delivery Date" TEXT,
        "Factory" TEXT
    )
    '''
    
    conn.execute(create_table_query)

    # Insert data into SQLite record by record
    cursor = conn.cursor()
    item = 1  # Initialize item counter
    
    for _, row in df.iterrows():
        sales_id = row['Sales ID']
        sales_name = row['Sales Name']
        customer_id = row['Customer ID']
        order_id = row['Order ID']
        yield_rate = float(row['Yield Rate']) if row['Yield Rate'] != '' else 0.0
        thru_put = float(row['Thru_put']) if row['Thru_put'] != '' else 0.0
        order_date = row['Order Date']
        delivery_date = row['Delivery Date']
        factory = row['Factory']

        # Insert each record one by one
        sql = f'''
        INSERT INTO {table_name} 
        ("Sales ID", "Sales Name", "Customer ID", "Order ID", "Yield Rate", "Thru_put", "Order Date", "Delivery Date", "Factory")
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        '''
        
        cursor.execute(sql, (sales_id, sales_name, customer_id, order_id, yield_rate, thru_put, order_date, delivery_date, factory))
        print(f"Inserted item {item}: {sales_id}, {sales_name}, {customer_id}, {order_id}")
        item += 1
    
    conn.commit()
    conn.close()
    print(f"Data has been successfully imported into '{db_filename}' database, table '{table_name}'.")

# Example usage
import pandas as pd

# Assuming 'df' is your DataFrame loaded from a CSV
df = pd.read_csv('sales_orders.csv')  # Replace 'sales_data.csv' with your actual file
save_to_sqlite(df)


Saving data to SQLite database record by record...
Inserted item 1: Sales A, Alice, Customer 16, 2024-45
Inserted item 2: Sales A, Alice, Customer 16, 2024-03
Inserted item 3: Sales A, Alice, Customer 16, 2024-12
Inserted item 4: Sales A, Alice, Customer 13, 2024-18
Inserted item 5: Sales A, Alice, Customer 13, 2024-03
Inserted item 6: Sales A, Alice, Customer 13, 2024-15
Inserted item 7: Sales A, Alice, Customer 13, 2024-38
Inserted item 8: Sales A, Alice, Customer 17, 2024-10
Inserted item 9: Sales A, Alice, Customer 17, 2024-16
Inserted item 10: Sales A, Alice, Customer 17, 2024-14
Inserted item 11: Sales A, Alice, Customer 17, 2024-29
Inserted item 12: Sales A, Alice, Customer 7, 2024-12
Inserted item 13: Sales A, Alice, Customer 7, 2024-22
Inserted item 14: Sales A, Alice, Customer 7, 2024-18
Inserted item 15: Sales A, Alice, Customer 20, 2024-17
Inserted item 16: Sales A, Alice, Customer 20, 2024-19
Inserted item 17: Sales A, Alice, Customer 20, 2024-10
Inserted item 18: Sales B,